In [1]:
from Bio import SeqIO
import csv
import pickle

In [2]:
chrom_lens = {"chr1": 248956422, 
             "chr2": 242193529, 
             "chr3": 198295559,
             "chr4": 190214555,
             "chr5": 181538259,
             "chr6": 170805979,
             "chr7": 159345973,
             "chr8": 145138636,
             "chr9": 138394717,
             "chr10": 133797422,
             "chr11": 135086622,
             "chr12": 133275309,
             "chr13": 114364328,
             "chr14": 107043718,
             "chr15": 101991189,
             "chr16": 90338345,
             "chr17": 83257441,
             "chr18": 80373285,
             "chr19": 58617616,
             "chr20": 64444167,
             "chr21": 46709983,
             "chr22": 50818468,
             "chrX": 156040895,
             "chrY": 57227415}

def pad_seq(g_seq, g_start, g_end):
    tholder = []
    g_start = int(g_start)
    g_end = int(g_end)
    if g_start < 150:
        g_start = 0
        g_end = g_end + 300-g_start
    elif g_end > chrom_lens[g_seq] - 150:
        g_end = chrom_lens[g_seq]
        g_start = g_start-300+chrom_lens[g_seq]-g_end
    else:
        g_start = g_start - 150
        g_end = g_end + 150
    tholder.append(g_seq)
    tholder.append(g_start)
    tholder.append(g_end)
    return tholder
# In[32]:

In [3]:
def dna_to_mrna_fwd(input_fasta):
    trslt_fasta = ''
    l = list(input_fasta)  
    for i in range(len(input_fasta)):
        if(l[i]=='G'):
            l[i]='C'
  
        elif(l[i]=='C'):
            l[i]='G'
  
        elif (l[i] == 'T'):
            l[i] = 'A'
  
        elif (l[i] == 'A'):
            l[i] = 'U'
            
        elif(l[i]=='g'):
            l[i]='c'   
            
        elif(l[i]=='c'):
            l[i]='g'
  
        elif (l[i] == 't'):
            l[i] = 'a'
  
        elif (l[i] == 'a'):
            l[i] = 'u'
    for char in l:
        trslt_fasta += char
    return trslt_fasta

def dna_to_mrna_rev(input_fasta):
    trslt_fasta = ''
    l = list(input_fasta)  
    for i in range(len(input_fasta)-1, -1, -1):
        if(l[i]=='G'):
            l[i]='C'
  
        elif(l[i]=='C'):
            l[i]='G'
  
        elif (l[i] == 'T'):
            l[i] = 'A'
  
        elif (l[i] == 'A'):
            l[i] = 'U'
            
        elif(l[i]=='g'):
            l[i]='c'   
            
        elif(l[i]=='c'):
            l[i]='g'
  
        elif (l[i] == 't'):
            l[i] = 'a'
  
        elif (l[i] == 'a'):
            l[i] = 'u'
    for char in reversed(l):
        trslt_fasta += char
    return trslt_fasta

In [5]:
'''def rewrite_bed_file(input_bed, output_file):
    namecount=0
    new_bed = open(output_file, 'w')
    with open(input_bed) as f:
        for line in f:
            L = line.strip().split()
            
            if L[0] == "chrM":
                continue 
                
            pholder = pad_seq(L[0], L[1], L[2])

            curr_seq = pholder[0]
            curr_start = str(pholder[1])
            curr_end = str(pholder[2])

            new_bed.write(curr_seq + '\t' + curr_start + '\t' + curr_end + '\t' + L[0] + '_' + str(L[5]) + '\n')
            namecount +=1
    return
'''

'def rewrite_bed_file(input_bed, output_file):\n    namecount=0\n    new_bed = open(output_file, \'w\')\n    with open(input_bed) as f:\n        for line in f:\n            L = line.strip().split()\n            \n            if L[0] == "chrM":\n                continue \n                \n            pholder = pad_seq(L[0], L[1], L[2])\n\n            curr_seq = pholder[0]\n            curr_start = str(pholder[1])\n            curr_end = str(pholder[2])\n\n            new_bed.write(curr_seq + \'\t\' + curr_start + \'\t\' + curr_end + \'\t\' + L[0] + \'_\' + str(L[5]) + \'\n\')\n            namecount +=1\n    return\n'

In [6]:
#rewrite_bed_file('Hela_data/Hela.bed', 'Hela_for_seq_extraction.bed')
#rewrite_bed_file('HEK293_data/HEK293.bed', 'HEK293_for_seq_extraction.bed')
#rewrite_bed_file('Hela_RIP_data/Hela_RIP.bed', 'Hela_RIP_for_seq_extraction.bed')
#rewrite_bed_file('HepG2_data/HepG2.bed', 'HepG2_for_seq_extraction.bed')

In [33]:
#read each seq in fasta file, put together tsv file

##read seqIO, get og_seq, split sample name to get chromosome and direction, transcribe accordingly, write to tsv using writer.
##create dictionary and pickle
def fasta_to_tsv(input_file, output_file, cell_type):
    namecount = 0
    new_seq_dict = {}
    with open(input_file, 'r') as f:
        with open(output_file, 'w') as outfile:
            seq_list = []
            writer = csv.writer(outfile, delimiter = "\t")
            for seq_record in SeqIO.parse(input_file, "fasta"):
                ##split into chromosome and dir
                tags = seq_record.id.strip().split('_')
                chrom = tags[0]
                ##get rid of ChrM
                if chrom == 'ChrM':
                    continue

                strand = tags[1]

                #create_name
                curr_name = cell_type + '_' + str(namecount)
                namecount+=1

                #trascribe seq
                og_seq = str(seq_record.seq)
                new_seq = ''

                pos_strand = True
                ##check strand direction
                if strand == '+':
                    pos_strand = True
                elif strand == '-':
                    pos_strand = False

                #transcribe sequence accordingly
                if (pos_strand == True):
                    new_seq = dna_to_mrna_fwd(og_seq)
                else:
                    new_seq = dna_to_mrna_rev(og_seq)
                
                seq_list.append[new_seq]
                ##eliminate repeat sequences from tsv so tsv and dictionary have same dimensions for model --> hasn't been run yet not sure if it matters
                if new_seq not in seq_list:
                    #have new sequence, have name, have chromosome, have strand (don't write to new file) --> not needed
                    writer.writerow([curr_name, new_seq, chrom])
                
                ##show status
                if namecount%5000 == 0:
                    print(namecount)
                    
        print(namecount)
    return    
            
            
            



In [34]:
fasta_to_tsv('HEK293_post_extraction.fa', 'sbp_polyA_RNA_seq_info/sbp_polyA_RNA_HEK293_mrna_seqs.tsv', 'HEK293')
fasta_to_tsv('Hela_post_extraction.fa', 'sbp_polyA_RNA_seq_info/sbp_polyA_RNA_Hela_mrna_seqs.tsv', 'Hela')
fasta_to_tsv('Hela_RIP_post_extraction.fa', 'sbp_polyA_RNA_seq_info/sbp_polyA_RNA_Hela_RIP_mrna_seqs.tsv', 'Hela_RIP')
fasta_to_tsv('HepG2_post_extraction.fa', 'sbp_polyA_RNA_seq_info/sbp_polyA_RNA_HepG2_mrna_seqs.tsv', 'HepG2')


5000
10000
12231
5000
10000
10889
5000
10000
15000
20000
25000
30000
31553
5000
10000
11251


In [ ]:
'''
5000
10000
12231
5000
10000
10889
5000
10000
15000
20000
25000
30000
31553
5000
10000
11251
'''